In [40]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, MaxPool2D, Flatten
import csv
# import pillow
import glob
# import pathlib



#defining paths to access files
train_images_dir = r"C:\Users\ESOMCHI\Downloads\planet deforestation data\planet\planet_deforestation_kaggle_comp\train-jpg"
test_images_dir = r"C:\Users\ESOMCHI\Downloads\planet deforestation data\planet\planet_deforestation_kaggle_comp\test-jpg"
extratest_images_dir = r"C:\Users\ESOMCHI\Downloads\planet deforestation data\test-jpg-additional"

# loading the csv file for the labels
label_path = r"C:\Users\ESOMCHI\Downloads\planet deforestation data\planet\planet_deforestation_kaggle_comp\train_classes.csv"
labels = pd.read_csv(label_path, index_col="image_name")
labellist = (labels.tags)

# encoding the labels using labelencoder
le = LabelEncoder()
labels = le.fit_transform(labels["tags"])



# calling image_loader function to load images in a particular directory
# train_images
x_train = keras.utils.image_dataset_from_directory(train_images_dir, image_size=(256, 256),
                                                   validation_split=0.2, subset="training", seed=123,
                                                       labels=labels[labels], label_mode='int', batch_size=32)
x_val = keras.utils.image_dataset_from_directory(train_images_dir, image_size=(256, 256),
                                                   validation_split=0.2, subset="validation", seed=123,
                                                       labels=labels[labels], label_mode='int', batch_size=32)
# test_images
x_test = keras.utils.image_dataset_from_directory(test_images_dir, image_size=(256, 256),
                                                       labels=None, label_mode='int', batch_size=32)
# extratest_images
xtra_test= keras.utils.image_dataset_from_directory(extratest_images_dir, image_size=(256, 256),
                                                            labels=None, label_mode='int', batch_size=32)




# splitting the labels set
y_train = labellist[:int((0.8 * len(labellist)))]
y_val = labellist[(int(0.8 * len(labellist))):]

# # reshaping the input datasets (the image dimensions)
# newdimension = np.square(train_images.shape[1])
# x_train = x_train.reshape(x_train.shape[0], newdimension)
# x_val = x_val.reshape(x_val.shape[0], newdimension)
# x_test = x_test.reshape(x_val.shape[0], newdimension)
# xtra_test = xtra_test.reshape(x_val.shape[0], newdimension)



# selecting and training a model
model_1 = Sequential(keras.layers.Rescaling(1./255))

model_1.add(Conv2D(128, (3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(12, 8)))
model_1.add(Dropout(.12))

model_1.add(MaxPool2D(100, (2, 2)))
model_1.add(Flatten())

model_1.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model_1.add(Dropout(0.2))

model_1.add(Dense(32, activation='softmax'))
model_1.add(Dropout(.24))

model_1.compile(optimizer="adam", loss="sparse_categorical_crossentropy", 
                metrics=["accuracy"])

model_1.fit(x_train, validation_data=(x_val, y_val), 
            batch_size=80, epochs=5, verbose=2)



# evaluating the model
loss, acc = model_1.evaluate(x_train, y_train, verbose=2)
print("accuracy: {} \n loss: {}".format(acc, loss))



# function to predict and transform back from encoded values to original texts
def pred_inverse_transform(test_dir):
  for pic in test_dir:
    taglist = np.array([])
    predict = model_1.predict(np.asarray(pic))
    tag_list = np.append(taglist, predict)
    return tag_list


tag_values = pred_inverse_transform(test_images_dir)
xtratag_values = pred_inverse_transform(extratest_images_dir)



# getting the names of the pictures and appending to a numpy array
for image_name in glob.glob(test_images_dir + "/*"):       #pathlib.join(test_images_dir, "/.jpg")):
  imagename = np.array(image_name)

for image_name in glob.glob(pathlib.join(extratest_images_dir, ".jpg")):
  xtra_imagename = np.array(image_name)



# writing back to csvfile
col = ["image_name", "tag"]

with open("test.csv", "w", encoding="UTF_8") as test:
  pd.to_csv(test, imagename, tag_values, columns=col)

with open("xtra_tes.csv", "w", encoding="UTF_8") as xtrat_test:
  pd.to_csv(xtra_test, xtra_imagename, xtratag_values, columns=col)

C:\Users\ESOMCHI\AppData\Roaming\Python\Python39\site-packages\keras\utils\image_dataset.py:163: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if labels not in ("inferred", None):


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()